# 1. 라이브러리 함수 호출

In [1]:
import pandas as pd 
import numpy as np
import pandas as pd
import numpy as np
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from lightgbm import plot_importance
import matplotlib.pyplot as plt
from lightgbm import LGBMRegressor
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier # model
from sklearn.preprocessing import LabelEncoder

# 2. 함수 설정

In [2]:




def label_code(dataframe,column):
    
    items=dataframe[column].tolist()
    encoder = LabelEncoder()
    encoder.fit(items)
    dataframe[column]=encoder.transform(items)
    return dataframe


def time_seperate(dataframe,column):
    y=list()
    m=list()
    d=list()
    for i in dataframe[column]:
        if i != 0:
            time= i.split('-')
            for j in range(len(time)):
                if j ==0:
                    y.append(int(time[j]))
                elif j ==1:
                    m.append(int(time[j]))
                elif j ==2:
                    d.append(int(time[j]))
        else:
            d.append(int(i))
            m.append(int(i))
            y.append(int(i))


    dataframe[column+'-year']=y
    dataframe[column+'-month']=m
    dataframe[column+'-day']=d
    return dataframe
def get_clf_eval(y_test, y_pred=None, pred_proba=None):

    accuracy = accuracy_score(y_test, y_pred)

    AUC = roc_auc_score(y_test, y_pred, pred_proba)

    print('오차행렬:\n', confusion)
    print('\n정확도: {:.4f}'.format(accuracy))



In [3]:
df=pd.read_csv('corporation_card_row.csv')

In [4]:
df=df.drop('Unnamed: 0',axis=1)
raw=df # 결과값 비교를 위해

In [5]:
df=df.drop(['최종 공제/불공제','코스트 센터','코스트센터명','위임자사번','원매입추심번호','카드사'],axis=1)
raw_lable=df['계정과목']

In [6]:
la=df.iloc[-8558:,:]


In [7]:
df[['결제(예정)일자','가맹점사업자번호','가맹점TEL','가맹점대표자','재직구분','고용형태코드','사용자사번']]=df[['결제(예정)일자','가맹점사업자번호','가맹점TEL','가맹점대표자','재직구분','고용형태코드','사용자사번']].fillna(0)



In [8]:
df=time_seperate(df,'결제(예정)일자')
df=time_seperate(df,'매입일자')
df=time_seperate(df,'승인(취소)일자')
df=df.drop(['결제(예정)일자','매입일자','승인(취소)일자'],axis=1)

In [9]:

df=df.fillna(0)


In [10]:
df_label=df['계정과목']
df=label_code(df,'계정과목')

In [11]:
label=df_label.value_counts()

In [12]:
encode_label=df['계정과목'].value_counts()


In [13]:
#가맹점TEL 전환
#업종코드 aaaa 0으로
df[df['가맹점TEL']=='--']
df.loc[df['가맹점TEL'] == '--','가맹점TEL'] = 0 
df[df['업종코드']=='AAAA']
df.loc[df['업종코드'] == 'AAAA','업종코드'] = 0 

In [14]:

for i in df:
    if df[i].dtype =='object':
        df=label_code(df,i)


# 모델선택

In [15]:
train_df=df.iloc[:-8558,:]
test_df=df.iloc[-8558:,:]

test_df=test_df.drop('계정과목',axis=1)
X_train_df=train_df.drop('계정과목',axis=1)



In [16]:
train_df=pd.get_dummies(train_df,columns=['계정과목'])


In [17]:
train_col=[]
for i in train_df:
    if '계정과목_' in i:
        train_col.append(i)
        

In [18]:
output_df_xg=test_df.copy()

In [19]:
y_train=train_df['계정과목_0']

# xgb


In [20]:

from hyperopt import hp

xgb_search_space = {
    'max_depth':hp.quniform('max_depth', 5, 20, 1),                     ## 정수형 하이퍼 파라미터 => quniform 사용
    'min_child_weight':hp.quniform('min_child_weight', 1, 2, 1),        ## 정수형 하이퍼 파라미터 => quniform 사용
    'learning_rate':hp.uniform('learning_rate', 0.01, 0.2),
    'colsample_bytree':hp.uniform('colsample_bytree', 0.5, 1),
}

In [21]:
from sklearn.model_selection import cross_val_score  ## 교차 검증
from xgboost import XGBClassifier
from hyperopt import STATUS_OK

def objective_func(search_space):
    xgb_clf = XGBClassifier(
      n_estimators=100,
      max_depth=int(search_space['max_depth']),                ## int형으로 형변환 필요
      min_child_weight=int(search_space['min_child_weight']),  ## int형으로 형변환 필요
      learning_rate=search_space['learning_rate'], 
      colsample_bytree=search_space['colsample_bytree'],
      eval_metric='logloss')
    accuracy = cross_val_score(xgb_clf, X_train_df, y_train, scoring='accuracy', cv=3)  ## 3개의 교차 검증 세트로 정확도 반환

  ## acc는 cv=3 개수만큼의 결과를 리스트로 가짐, 이를 평균하여 반환하되, -1을 곱함
    return {
      'loss':(-1) * np.mean(accuracy),
      'status':STATUS_OK}

In [22]:
from hyperopt import fmin, tpe, Trials

trial_val = Trials() ## 결과 저장

best = fmin(
    fn=objective_func,
    space=xgb_search_space,
    algo=tpe.suggest,
    max_evals=50,    ## 최대 반복 횟수 지정
    trials=trial_val,
)


100%|████████| 50/50 [00:59<00:00,  1.19s/trial, best loss: -0.9999044342507645]


In [23]:
best

{'colsample_bytree': 0.9281669127762595,
 'learning_rate': 0.08576227182490234,
 'max_depth': 7.0,
 'min_child_weight': 1.0}

In [24]:
xgb_wrapper = XGBClassifier(
    n_estimators=400,
    learning_rate=round(best['learning_rate'], 5),
    max_depth=int(best['max_depth']),
    min_child_weight=int(best['min_child_weight']),
    colsample_bytree=round(best['colsample_bytree'], 5)
)


In [25]:

output_df_xg['result']=0
for i in train_col:
    y_train=train_df[i]
    xgb_wrapper.fit(X_train_df, y_train)
    pre=xgb_wrapper.predict(test_df)
    output_df_xg[i]=pre
    

# 복수 측정된 값 찾아서 처리하기


In [26]:
for i in output_df_xg.index:
    row=output_df_xg.loc[i]
    subject_codes=[]
    for j in row.index:
        if '계정과목_' in j:
            if row[j]==1:
                subject_codes.append(j.split('_')[1])
    if len(subject_codes)==0:
        output_df_xg.loc[i,'result']=999
    elif len(subject_codes)==1:
        output_df_xg.loc[i,'result']=subject_codes[0]
    elif len(subject_codes)>=2:
        output_df_xg.loc[i,'result']=','.join(subject_codes)
            



In [27]:
output_df_xg['result'].unique()

array(['24', '44', '10', '5', '1', '16', '2', '22', '13', '14', '55',
       '11', '41', '59', '62', '17', '54', '4', '50', '32', '48', '49',
       '42', '21', '56', '36', '33', '3', 999, '37', '46', '65', '39',
       '47', '63', '20', '27', '25', '38', '32,33', '66', '18', '65,66'],
      dtype=object)

# 데이터 프레임 라벨링 코딩한거 다니 역치환

In [28]:
for i in output_df_xg:
    if '계정과목_' in i:
        #split_col=i.split('_')
        #output_df_xg.loc[output_df_xg[i] == 1,'result'] = int(split_col[1])
        output_df_xg=output_df_xg.drop(i,axis=1)


In [29]:
data=output_df_xg.copy()
data

,소유자명,사용자사번,고용형태코드,재직구분,카드번호,승인번호,승인구분,법인카드증빙일련번호,승인시간(취소),총금액,...,결제(예정)일자-year,결제(예정)일자-month,결제(예정)일자-day,매입일자-year,매입일자-month,매입일자-day,승인(취소)일자-year,승인(취소)일자-month,승인(취소)일자-day,result
20926,244,2023,2,1,5589789900001096,55862687,0,20866,2234,47320,...,2022,3,23,2022,2,12,2022,2,11,24
20927,1256,229,2,1,5589789900002136,46857956,0,20867,6205,71000,...,2022,3,23,2022,2,12,2022,2,11,44
20928,1300,1007,2,1,5531760001505755,77997353,0,20868,11550,265000,...,2022,3,23,2022,2,12,2022,2,11,44
20929,384,1868,2,1,5531760001062732,61561484,0,20869,4465,13000,...,2022,3,23,2022,2,12,2022,2,11,10
20930,1390,1560,2,1,5531760000632246,3634825,0,20870,6104,76200,...,2022,3,23,2022,2,12,2022,2,11,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29479,1123,285,2,1,5105540000259786,67270015,0,29367,7139,9200,...,2022,3,23,2022,3,1,2022,2,28,10
29480,2416,734,2,1,5105540000260446,67194022,0,29368,1698,28800,...,2022,3,23,2022,3,2,2022,2,28,24
29481,1355,749,2,1,5105540000263820,43028048,0,29369,6466,103000,...,2022,3,23,2022,3,1,2022,2,28,10
29482,1752,765,2,1,5105540000264349,24225060,0,29370,16250,12500,...,2022,3,23,2022,3,1,2022,2,28,2


In [30]:
output_df_xg=data
temp=[0 for _ in range(len(encode_label))]

for i in range(len(encode_label.index)):
    temp[encode_label.index[i]]=label.index[i]
    

for i in output_df_xg.index:
    x=output_df_xg.loc[i]['result']
    x=str(x)
    if ',' in x:
        print(x)
        '''
        temp_re=x.split(',')
        temp_=[]
        for i in temp_re:
            temp_.append(str(temp[int(i)]))
        print(i,temp_)
        output_df_xg.loc[i,'result']=','.join(temp_) 
        '''
    elif '999' in x:
        print(x)
    else:
        output_df_xg.loc[i,'result']=str(temp[int(x)])

999
999
999
999
999
999
999
999
999
999
999
999
999
999
999
999
999
999
999
999
999
999
999
999
999
999
999
32,33
32,33
999
999
999
999
999
999
999
999
65,66
65,66
65,66


In [31]:
output_df_xg['result'].unique()

array(['530307050', '531701010', '530301210', '530301080', '530301010',
       '530305030', '530301020', '530307020', '530305001', '530305010',
       '531719020', '530301250', '530781775', '531725001', '531725775',
       '530305040', '531719001', '530301050', '531707775', '530725020',
       '531707001', '531707020', '530901010', '530307010', '531719775',
       '530781010', '530725030', '530301030', 999, '530781030',
       '531703010', '531729020', '530781070', '531705020', '531727010',
       '530307001', '530717010', '530307060', '530781050', '32,33',
       '531729775', '530305050', '65,66'], dtype=object)

In [38]:
raw_label

NameError: name 'raw_label' is not defined

# 결과


In [32]:
temp[65]

531729020

In [33]:
temp[66]

531729775

In [34]:
output_df_xg.to_csv('result_xg.csv')

In [35]:
output_df.to_csv('result.csv')

NameError: name 'output_df' is not defined

In [ ]:
x=la['계정과목']

In [ ]:
y=output_df_xg['result']

In [ ]:
error=pd.DataFrame(x-y,columns=['error'])

In [ ]:
100-(len(error[error['error']!=0])/len(error['error']))*100

In [ ]:
len(error[error['error']!=0])

# 에러 잡는법 생각하기

In [ ]:
result=[]
for i in output_df.index:
    x=output_df.loc[i]
    temp=[]
    for j in range(len(x)):
        if j >= 40:
            temp.append(x[j])
    result.append(temp)
count=[]
for i in range(len(result)):
    value=0
    for j in range(len(result[0])):
        if result[i][j]==1.0:
            value+=1
    count.append(value)
        
#40~106

v=[]
for i in range(len(count)):
    if count[i] >=2:
        if count[i]>=3:
            print(i,'!!!')
        v.append(i)

        
        
error_=output_df.iloc[v]
for i in error_:
    if '계정과목_' in i:
        